## Drugbank Crawler
### Search by target name and download drug name, SMILES, and URL.

In [1]:
from bs4 import BeautifulSoup
import urllib
from urllib.parse import urlparse, parse_qs, quote
import requests
import pandas as pd

headers={'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.142 Safari/537.36'}

In [2]:
# https://stackoverflow.com/questions/2506379/add-params-to-given-url-in-python
def change_parameters(url, params):
    url_parts = list(urllib.parse.urlparse(url))
    query = dict(urllib.parse.parse_qsl(url_parts[4]))
    query.update(params)
    url_parts[4] = urllib.parse.urlencode(query)
    URL = urllib.parse.urlunparse(url_parts)
    return URL

In [3]:
def drug_crawl(target_name="D(2) dopamine receptor"):
    url = "https://go.drugbank.com/targets"
    target_params = {'utf8': ['✓'],
     'approved': ['0'],
     'nutraceutical': ['0'],
     'illicit': ['0'],
     'investigational': ['0'],
     'withdrawn': ['0'],
     'experimental': ['0'],
     'us': ['0'],
     'ca': ['0'],
     'eu': ['0'],
     'q[polypeptides.name]': target_name}

    URL = change_parameters(url, target_params)
    print(URL)
    resp = requests.request("get", URL, params=None, data=None, headers=headers)
    drug_url_list = []
    try:
        pages = int(parse_qs(BeautifulSoup(resp.content).find("li", "page-item last").find('a', href=True)['href'])["page"][0])
        for page in range(pages):
            page_url = change_parameters(URL, {'page': [page+1]})
            resp = requests.request("get", page_url, params=None, data=None, headers=headers)
            for tr in BeautifulSoup(resp.content).find('tbody').find_all("tr"):
                drug_url_list.append(urllib.parse.urljoin("https://go.drugbank.com/", tr.find("a", href=True)['href']))
        
    except:
        for tr in BeautifulSoup(resp.content).find('tbody').find_all("tr"):
            drug_url_list.append(urllib.parse.urljoin("https://go.drugbank.com/", tr.find("a", href=True)['href']))
    
    print("drug_list : ", len(drug_url_list))
        
    final_list = []
    for u in drug_url_list:
        resp = requests.request("get", u, params=None, data=None, headers=headers)
        try:
            sm = BeautifulSoup(resp.content).find(id="smiles").nextSibling.get_text()
            print(sm)
            name = BeautifulSoup(resp.content).find("h1").get_text()
            print(name)
            final_list.append((name, sm, u))
        except:
            pass
    print(f"small molecules :{len(final_list)}")
    pd.DataFrame(final_list, columns =['name', 'smiles', 'url']).to_csv(f"{target_name}.csv", index=False)
    return
          
        
    

In [4]:
drug_crawl("Fatty acid synthase")

https://go.drugbank.com/targets?utf8=%5B%27%E2%9C%93%27%5D&approved=%5B%270%27%5D&nutraceutical=%5B%270%27%5D&illicit=%5B%270%27%5D&investigational=%5B%270%27%5D&withdrawn=%5B%270%27%5D&experimental=%5B%270%27%5D&us=%5B%270%27%5D&ca=%5B%270%27%5D&eu=%5B%270%27%5D&q%5Bpolypeptides.name%5D=Fatty+acid+synthase
drug_list :  5
[H][C@]1(O[C@]1([H])C(=O)CC\C=C\C\C=C\C)C(N)=O
Cerulenin
CCCCCCCCCCC[C@@H](C[C@@H]1OC(=O)[C@H]1CCCCCC)OC(=O)[C@H](CC(C)C)NC=O
Orlistat
CCCCCCCCCCC[C@@H](C[C@@H]1OC(=O)[C@H]1CCCCCC)OC(=O)[C@H](CC(C)C)NC=O
Orlistat
NC(=O)C1=NC=CN=C1
Pyrazinamide
[O-][N+](=O)C1=CN2C[C@@H](COC2=N1)OCC1=CC=C(OC(F)(F)F)C=C1
Pretomanid
small molecules :5
